In [1]:
# Import the requests library.
import requests
import numpy as np
import pandas as pd
# Import the requests library.
import requests
import datetime
from datetime import datetime
import matplotlib.pyplot as plt
# Dependencies for the wine API
import urllib
import json
# Import the API key.
from config import Token_NOAA
from config import API_Token
import calendar
#Suppress Warnings
pd.options.mode.chained_assignment = None  # default='warn'
#Display all columns
pd.set_option('display.max_columns', None)

# statistics
from scipy.stats import linregress
from scipy import stats
import statistics as st
import matplotlib.pyplot as plt

# Wine and Weather Data

#### Import CSV Files

In [2]:
temp_df=pd.read_csv('Final_Temp_Weather_API.csv')
prcp_df=pd.read_csv('Final_Prcp_Weather_API.csv')
red_USA_df=pd.read_csv('Red_Wine_USA.csv', index_col=0)
white_USA_df=pd.read_csv('White_Wine_USA.csv',index_col=0)

In [3]:
red_USA_df

,wine,wine_id,appellation,color,wine_type,regions,country,classification,vintage,is_primeurs,score,confidence_index,journalist_count
0,"Chateau Montelena, Cabernet Sauvignon, Napa Va...",39440,Napa Valley,Red,NaN,California,Usa,NaN,1991,False,94.78,C+,4
1,"Ridge Vineyards, Monte Bello, Santa Cruz Mount...",120786,Santa Cruz Mountains,Red,NaN,California,Usa,NaN,2013,False,95.83,C+,8
2,"Quintessa, Rutherford",118360,Rutherford,Red,NaN,California,Usa,NaN,2015,False,95.77,C,3
3,"Harlan Estate, Napa Valley",81980,Napa Valley,Red,NaN,California,Usa,NaN,2001,False,98.89,A+,6
4,"Dalla Valle Vineyards, Maya, Napa Valley",48290,Napa Valley,Red,NaN,California,Usa,NaN,2001,False,96.49,B+,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4359,"Pine Ridge Vineyards, Rutherford Cabernet Sauv...",116343,Napa Valley,Red,NaN,California,Usa,NaN,1995,False,85.97,C+,3
4360,"La Jota Vineyard, Cabernet Sauvignon, Howell M...",92780,Howell Mountain,Red,NaN,California,Usa,NaN,1995,False,89.51,C,3
4361,"Pine Ridge Vineyards, Stags Leap District Cabe...",116344,Napa Valley,Red,NaN,California,Usa,NaN,1995,False,84.59,C,3
4362,"Flora Springs, Sangiovese, Napa Valley",74746,Napa Valley,Red,NaN,California,Usa,NaN,1996,False,85.51,C,3


In [4]:
white_USA_df

,wine,wine_id,appellation,color,wine_type,regions,country,classification,vintage,is_primeurs,score,confidence_index,journalist_count
0,"Marcassin Estate, Chardonnay, White, Sonoma Coast",101634,Sonoma Coast,White,dry,California,Usa,NaN,2009,False,93.08,C+,3
1,"Ridge Vineyards, Monte Bello Chardonnay, White...",120785,Santa Cruz Mountains,White,dry,California,Usa,NaN,2016,False,94.39,B,3
2,"Stags' Leap Winery, Chardonnay, White, Napa Va...",129823,Napa Valley,White,dry,California,Usa,NaN,2016,False,89.83,A+,3
3,"Joseph Phelps Vineyards, Freestone Chardonnay,...",89569,Sonoma Coast,White,dry,California,Usa,NaN,2016,False,90.73,C+,4
4,"Crossbarn By Paul Hobbs, Chardonnay, White, So...",47377,Sonoma Coast,White,dry,California,Usa,NaN,2016,False,90.39,B+,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
779,"Peter Michael Winery, Belle Cote Chardonnay, W...",114819,Sonoma County,White,dry,California,Usa,NaN,1996,False,91.76,C+,3
780,"Peter Michael Winery, Clos Du Ciel Chardonnay,...",114820,California,White,dry,California,Usa,NaN,1996,False,91.00,C+,3
781,"Kistler Vineyards, Hudson Vineyard Chardonnay,...",91298,Carneros,White,dry,California,Usa,NaN,1995,False,93.90,A,3
782,"Joseph Phelps Vineyards, Carneros Chardonnay, ...",89562,Los Carneros,White,dry,California,Usa,NaN,1996,False,86.86,A,3


In [5]:
calender_years=[1992,1993, 1994, 1995,1996,1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,2013, 2014,2015, 2016, 2017]
calender_months=['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

# Combine Weather Data and White Wine Data

In [6]:
#Create empty columns to iterate through
for month in calender_months:
    white_USA_df['avgPrcp'+str(month)]=0.00
    white_USA_df['avgTemp'+str(month)]=0
#Combine white wine with avgerage temperature and precipitation data
for i in range(len(white_USA_df)):
    for month in calender_months: 
        for year in calender_years:

            if white_USA_df['vintage'][i]==year:
#------------------------------------------------------------------California-------------------------------------------------- 
                if white_USA_df['appellation'][i]=='Napa Valley' or white_USA_df['appellation'][i]=='Los Carneros' or white_USA_df['appellation'][i]=='Atlas Peak' or white_USA_df['appellation'][i]=='Calistoga' or white_USA_df['appellation'][i]=='Napa County' or white_USA_df['appellation'][i]=='Rutherford' or white_USA_df['appellation'][i]=='Oakville' or white_USA_df['appellation'][i]=='Howell Mountain' or white_USA_df['appellation'][i]=='Stags Leap District' or white_USA_df['appellation'][i]=='St Helena' or white_USA_df['appellation'][i]=='Knights Valley' or white_USA_df['appellation'][i]=='Oak Knoll District' or white_USA_df['appellation'][i]=='Coombsville' or white_USA_df['appellation'][i]=='Spring Mountain District' or white_USA_df['appellation'][i]=='Mount Veeder' or white_USA_df['appellation'][i]=='Yountville' or white_USA_df['appellation'][i]=='Diamond Mountain District' or white_USA_df['appellation'][i]=='Green Valley':
                    try:
                        white_USA_df['avgPrcp'+month][i]=prcp_df.loc[prcp_df['months']==month].loc[prcp_df['year']==year].PRCPNapa
                        white_USA_df['avgTemp'+month][i]=temp_df.loc[temp_df['Months']==month].loc[temp_df['Year']==year].avgTempNapa
                    except:
                        pass
                if white_USA_df['appellation'][i]=='Santa Cruz Mountains' or white_USA_df['appellation'][i]=='Santa Lucia Highlands' or white_USA_df['appellation'][i]=='Santa Ynez Valley' or white_USA_df['appellation'][i]=='St Rita Hills':
                    try:
                        white_USA_df['avgPrcp'+month][i]=prcp_df.loc[prcp_df['months']==month].loc[prcp_df['year']==year].PRCPSanta
                        white_USA_df['avgTemp'+month][i]=temp_df.loc[temp_df['Months']==month].loc[temp_df['Year']==year].avgTempSanta
                    except:
                        pass
                if white_USA_df['appellation'][i]=='Sonoma Coast' or white_USA_df['appellation'][i]=='Chalk Hill' or white_USA_df['appellation'][i]=='Sonoma Valley' or white_USA_df['appellation'][i]=='Bennett Valley' or white_USA_df['appellation'][i]=='Mendocino Ridge' or white_USA_df['appellation'][i]=='Russian River Valley' or white_USA_df['appellation'][i]=='Sonoma Coast' or white_USA_df['appellation'][i]=='Carneros' or white_USA_df['appellation'][i]=='Alexander Valley' or white_USA_df['appellation'][i]=='Dry Creek Valley':
                    try:
                        white_USA_df['avgPrcp'+month][i]=prcp_df.loc[prcp_df['months']==month].loc[prcp_df['year']==year].PRCPSonoma
                        white_USA_df['avgTemp'+month][i]=temp_df.loc[temp_df['Months']==month].loc[temp_df['Year']==year].avgTempSonoma
                    except:
                        pass
#------------------------------------------------------------------Washington--------------------------------------------------
                if white_USA_df['appellation'][i]=='Yakima Valley' or white_USA_df['appellation'][i]=='Horse Heaven Hills' or white_USA_df['appellation'][i]=='Snipes Mountains' or white_USA_df['appellation'][i]=='Red Mountain':
                    try:
                        white_USA_df['avgPrcp'+month][i]=prcp_df.loc[prcp_df['months']==month].loc[prcp_df['year']==year].PRCPYakima
                        white_USA_df['avgTemp'+month][i]=temp_df.loc[temp_df['Months']==month].loc[temp_df['Year']==year].avgTempYakima
                    except:
                        pass
                if white_USA_df['appellation'][i]=='Walla Walla Valley':
                    try:
                        white_USA_df['avgPrcp'+month][i]=prcp_df.loc[prcp_df['months']==month].loc[prcp_df['year']==year].PRCPWalla
                        white_USA_df['avgTemp'+month][i]=temp_df.loc[temp_df['Months']==month].loc[temp_df['Year']==year].avgTempWalla
                    except:
                        pass
                if white_USA_df['appellation'][i]=='Columbia Valley':
                    try:
                        white_USA_df['avgPrcp'+month][i]=prcp_df.loc[prcp_df['months']==month].loc[prcp_df['year']==year].PRCPColumbia
                        white_USA_df['avgTemp'+month][i]=temp_df.loc[temp_df['Months']==month].loc[temp_df['Year']==year].avgTempColumbia
                    except:
                        pass
#------------------------------------------------------------------Oregon--------------------------------------------------
                if white_USA_df['appellation'][i]=='Willamette Valley' or white_USA_df['appellation'][i]=='Dundee Hills' or white_USA_df['appellation'][i]=='Eola-Amity Hills' or white_USA_df['appellation'][i]=='Ribbon Ridge' or white_USA_df['appellation'][i]=='Chehalem Mountains':
                    try:
                        white_USA_df['avgPrcp'+month][i]=prcp_df.loc[prcp_df['months']==month].loc[prcp_df['year']==year].PRCPDundee
                        white_USA_df['avgTemp'+month][i]=temp_df.loc[temp_df['Months']==month].loc[temp_df['Year']==year].avgTempDundee
                    except:
                        pass

# Combine Weather Data and Red Wine Data

In [7]:
#Create empty columns to iterate through
for month in calender_months:
    red_USA_df['avgPrcp'+str(month)]=0.00
    red_USA_df['avgTemp'+str(month)]=0
#Combine white wine with avgerage temperature and precipitation data
for i in range(len(red_USA_df)):
    for month in calender_months:
        for year in calender_years:
            
            if red_USA_df['vintage'][i]==year:
#------------------------------------------------------------------California-------------------------------------------------- 
                if red_USA_df['appellation'][i]=='Napa Valley' or red_USA_df['appellation'][i]=='Los Carneros' or red_USA_df['appellation'][i]=='Atlas Peak' or red_USA_df['appellation'][i]=='Calistoga' or red_USA_df['appellation'][i]=='Napa County' or red_USA_df['appellation'][i]=='Rutherford' or red_USA_df['appellation'][i]=='Oakville' or red_USA_df['appellation'][i]=='Howell Mountain' or red_USA_df['appellation'][i]=='Stags Leap District' or red_USA_df['appellation'][i]=='St Helena' or red_USA_df['appellation'][i]=='Knights Valley' or red_USA_df['appellation'][i]=='Oak Knoll District' or red_USA_df['appellation'][i]=='Coombsville' or red_USA_df['appellation'][i]=='Spring Mountain District' or red_USA_df['appellation'][i]=='Mount Veeder' or red_USA_df['appellation'][i]=='Yountville' or red_USA_df['appellation'][i]=='Diamond Mountain District' or red_USA_df['appellation'][i]=='Green Valley':
                    try:
                        red_USA_df['avgPrcp'+month][i]=prcp_df.loc[prcp_df['months']==month].loc[prcp_df['year']==year].PRCPNapa
                        red_USA_df['avgTemp'+month][i]=temp_df.loc[temp_df['Months']==month].loc[temp_df['Year']==year].avgTempNapa
                    except:
                        pass
                if red_USA_df['appellation'][i]=='Santa Cruz Mountains' or red_USA_df['appellation'][i]=='Santa Lucia Highlands' or red_USA_df['appellation'][i]=='Santa Ynez Valley' or red_USA_df['appellation'][i]=='St Rita Hills':
                    try:
                        red_USA_df['avgPrcp'+month][i]=prcp_df.loc[prcp_df['months']==month].loc[prcp_df['year']==year].PRCPSanta
                        red_USA_df['avgTemp'+month][i]=temp_df.loc[temp_df['Months']==month].loc[temp_df['Year']==year].avgTempSanta
                    except:
                        pass
                if red_USA_df['appellation'][i]=='Sonoma Coast' or red_USA_df['appellation'][i]=='Chalk Hill' or red_USA_df['appellation'][i]=='Sonoma Valley' or red_USA_df['appellation'][i]=='Bennett Valley' or red_USA_df['appellation'][i]=='Mendocino Ridge' or red_USA_df['appellation'][i]=='Russian River Valley' or red_USA_df['appellation'][i]=='Sonoma Coast' or red_USA_df['appellation'][i]=='Carneros' or red_USA_df['appellation'][i]=='Alexander Valley' or red_USA_df['appellation'][i]=='Dry Creek Valley':
                    try:
                        red_USA_df['avgPrcp'+month][i]=prcp_df.loc[prcp_df['months']==month].loc[prcp_df['year']==year].PRCPSonoma
                        red_USA_df['avgTemp'+month][i]=temp_df.loc[temp_df['Months']==month].loc[temp_df['Year']==year].avgTempSonoma
                    except:
                        pass
                if red_USA_df['appellation'][i]=='Yakima Valley' or red_USA_df['appellation'][i]=='Horse Heaven Hills' or red_USA_df['appellation'][i]=='Snipes Mountains' or red_USA_df['appellation'][i]=='Red Mountain':
                    try:
                        red_USA_df['avgPrcp'+month][i]=prcp_df.loc[prcp_df['months']==month].loc[prcp_df['year']==year].PRCPYakima
                        red_USA_df['avgTemp'+month][i]=temp_df.loc[temp_df['Months']==month].loc[temp_df['Year']==year].avgTempYakima
                    except:
                        pass
                if red_USA_df['appellation'][i]=='Walla Walla Valley':
                    try:
                        red_USA_df['avgPrcp'+month][i]=prcp_df.loc[prcp_df['months']==month].loc[prcp_df['year']==year].PRCPWalla
                        red_USA_df['avgTemp'+month][i]=temp_df.loc[temp_df['Months']==month].loc[temp_df['Year']==year].avgTempWalla
                    except:
                        pass
                if red_USA_df['appellation'][i]=='Columbia Valley':
                    try:
                        red_USA_df['avgPrcp'+month][i]=prcp_df.loc[prcp_df['months']==month].loc[prcp_df['year']==year].PRCPColumbia
                        red_USA_df['avgTemp'+month][i]=temp_df.loc[temp_df['Months']==month].loc[temp_df['Year']==year].avgTempColumbia
                    except:
                        pass
                if red_USA_df['appellation'][i]=='Willamette Valley' or red_USA_df['appellation'][i]=='Dundee Hills' or red_USA_df['appellation'][i]=='Eola-Amity Hills' or red_USA_df['appellation'][i]=='Ribbon Ridge' or red_USA_df['appellation'][i]=='Chehalem Mountains':
                    try:
                        red_USA_df['avgPrcp'+month][i]=prcp_df.loc[prcp_df['months']==month].loc[prcp_df['year']==year].PRCPDundee
                        red_USA_df['avgTemp'+month][i]=temp_df.loc[temp_df['Months']==month].loc[temp_df['Year']==year].avgTempDundee
                    except:
                        pass

# red_USA_df       
# sorted_red=red_USA_df[red_USA_df['vintage']==2016]
# sorted_red

##### Print Raw Combined White Wine/Weather and Red Wine/Weather

In [8]:
white_USA_df.to_csv('Final_White_Temp_Prcp.csv')
red_USA_df.to_csv('Final_Red_Temp_Prcp.csv')

# Cleaning Red Wine Weather and White Wine Weather

## Red Wine Weather

#### Print the number of NA values

In [9]:
print('Red Wine')
print('______________________________')
print(red_USA_df.isnull().sum())


Red Wine
______________________________
wine                   0
wine_id                0
appellation            0
color                  0
wine_type           4352
regions                0
country                0
classification      4364
vintage                0
is_primeurs            0
score                  0
confidence_index       0
journalist_count       0
avgPrcpJanuary       239
avgTempJanuary         0
avgPrcpFebruary        0
avgTempFebruary        0
avgPrcpMarch         182
avgTempMarch           0
avgPrcpApril           0
avgTempApril           0
avgPrcpMay           194
avgTempMay             0
avgPrcpJune         2369
avgTempJune            0
avgPrcpJuly         2750
avgTempJuly            0
avgPrcpAugust       3793
avgTempAugust          0
avgPrcpSeptember    3793
avgTempSeptember       0
avgPrcpOctober      3793
avgTempOctober         0
avgPrcpNovember     3793
avgTempNovember        0
avgPrcpDecember     3920
avgTempDecember        0
dtype: int64


#### Determine the number of NA values for each appellations

In [10]:
a=[]
b=[]
c=[]
d=[]
e=[]
f=[]
g=[]
h=[]
wanted_napa=['Napa Valley', 'Los Carneros', 'Atlas Peak', 'Calistoga', 'Napa County', 'Rutherford', 'Oakville', 'Howell Mountain', 'Stags Leap District', 'St Helena', 'Knights Valley', 'Oak Knoll District', 'Coombsville', 'Spring Mountain District', 'Mount Veeder', 'Yountville', 'Diamond Mountain District', 'Green Valley']
wanted_santa=['Santa Cruz Mountains', 'Sta Rita Hills','Santa Lucia Highlands', 'Santa Ynez Valley','St Rita Hills']
wanted_sonoma=['Sonoma Coast', 'Chalk Hill', 'Sonoma Valley', 'Bennett Valley', 'Mendocino Ridge', 'Russian River Valley', 'Sonoma Coast', 'Carneros', 'Alexander Valley', 'Dry Creek Valley', 'Sonoma County', 'Sonoma Mountain']
wanted_yakima=['Yakima Valley','Horse Heaven Hills','Snipes Mountains', 'Red Mountain']
wanted_walla=['Walla Walla Valley']
wanted_columbia=['Columbia Valley']
wanted_will=['Willamette Valley', 'Dundee Hills', 'Eola-Amity Hills', 'Ribbon Ridge', 'Chehalem Mountains']
wanted_others= wanted_napa +wanted_santa+wanted_sonoma+wanted_yakima+wanted_walla+wanted_columbia+wanted_will
for value in red_USA_df['appellation']:
    if value in wanted_napa: 
        a.append(value)
    if value in wanted_santa: 
        b.append(value)
    if value in wanted_sonoma: 
        c.append(value)
    if value in wanted_yakima: 
        d.append(value)
    if value in wanted_walla: 
        e.append(value)
    if value in wanted_columbia: 
        f.append(value)
    if value in wanted_will: 
        g.append(value)
    if value not in wanted_others:
        red_USA_df.drop(red_USA_df.loc[red_USA_df['appellation']==value].index, inplace= True)
        h.append(value)
        
print('Napa Valley')
print(len(a))

print('Santa Cruz Mountains')
print(len(b))

print('Sonoma Coast')
print(len(c))

print('Yakima Valley')
print(len(d))

print('Walla Walla')
print(len(e))

print('Columbia')
print(len(f))

print('Willamette Valley')
print(len(g))
        

print('Others')
print(len(h))
print(h)

Napa Valley
3030
Santa Cruz Mountains
76
Sonoma Coast
432
Yakima Valley
167
Walla Walla
271
Columbia
212
Willamette Valley
97
Others
79
['Amador County', 'California', 'Amador County', 'California', 'Oregon', 'Oregon', 'Paso Robles', 'California', 'Monterey', 'California', 'California', 'Anderson Valley', 'Paso Robles', 'California', 'California', 'California', 'California', 'California', 'California', 'Mount Harlan', 'California', 'Oregon', 'California', 'Paso Robles', 'Paso Robles', 'California', 'California', 'Washington', 'Monticello', 'Washington', 'Oregon', 'California', 'California', 'California', 'California', 'Paso Robles', 'California', 'Oregon', 'Washington', 'California', 'California', 'California', 'Washington', 'Oregon', 'Paso Robles', 'California', 'Paso Robles', 'California', 'Washington', 'California', 'Amador County', 'California', 'Anderson Valley', 'Washington', 'California', 'California', 'Oregon', 'California', 'California', 'California', 'Paso Robles', 'Paso Robl

#### Determine number of NA values for each State

In [11]:
cali=['California']
oregon=['Oregon']
wash=['Washington']
total=['California','Oregon', 'Washington']
x=[]
y=[]
z=[]
xyz=[]

for value in red_USA_df['regions']:
    if value in cali:
        x.append(value)
    if value in oregon:
        y.append(value)
    if value in wash:
        z.append(value)
    if value not in total:
        xyz.append(value)
print('California')
print(red_USA_df['regions'].str.count('California').sum())
print('Oregon')
print(red_USA_df['regions'].str.count('Oregon').sum())
print('Washington')
print(red_USA_df['regions'].str.count('Washington').sum())
print("Other")
print(len(xyz))

California
3538
Oregon
97
Washington
650
Other
0


#### Determine which columns have the most NA values

In [12]:
print(red_USA_df['wine_type'].isnull().sum())
print(red_USA_df['classification'].isnull().sum())
print(red_USA_df['avgPrcpJanuary'].isnull().sum())
print(white_USA_df['avgPrcpJune'].isnull().sum())
print(red_USA_df['avgPrcpSeptember'].isnull().sum())
print(red_USA_df['avgPrcpOctober'].isnull().sum())
print(red_USA_df['avgPrcpNovember'].isnull().sum())
print(red_USA_df['avgPrcpDecember'].isnull().sum())

4273
4285
239
399
3793
3793
3793
3920


#### Statistics for Red Wine

In [13]:
print(st.mean(red_USA_df['score']))
print(st.median(red_USA_df['score']))
print(st.variance(red_USA_df['score']))
print(st.stdev(red_USA_df['score']))

92.01515752625437
92.03
8.624524234049902
2.9367540302262127


## White Wine

##### Print the number of NA values

In [14]:
print('White Wine')
print('______________________________')
print(white_USA_df.isnull().sum())

White Wine
______________________________
wine                  0
wine_id               0
appellation           0
color                 0
wine_type             2
regions               0
country               0
classification      784
vintage               0
is_primeurs           0
score                 0
confidence_index      0
journalist_count      0
avgPrcpJanuary       24
avgTempJanuary        0
avgPrcpFebruary       0
avgTempFebruary       0
avgPrcpMarch         32
avgTempMarch          0
avgPrcpApril          0
avgTempApril          0
avgPrcpMay           27
avgTempMay            0
avgPrcpJune         399
avgTempJune           0
avgPrcpJuly         468
avgTempJuly           0
avgPrcpAugust       621
avgTempAugust         0
avgPrcpSeptember    621
avgTempSeptember      0
avgPrcpOctober      621
avgTempOctober        0
avgPrcpNovember     621
avgTempNovember       0
avgPrcpDecember     643
avgTempDecember       0
dtype: int64


#### Determine number of instances of appellations

In [15]:
a=[]
b=[]
c=[]
d=[]
e=[]
f=[]
g=[]
h=[]
wanted_napa=['Napa Valley', 'Los Carneros', 'Atlas Peak', 'Calistoga', 'Napa County', 'Rutherford', 'Oakville', 'Howell Mountain', 'Stags Leap District', 'St Helena', 'Knights Valley', 'Oak Knoll District', 'Coombsville', 'Spring Mountain District', 'Mount Veeder', 'Yountville', 'Diamond Mountain District', 'Green Valley']
wanted_santa=['Santa Cruz Mountains', 'Santa Lucia Highlands', 'Santa Ynez Valley','St Rita Hills']
wanted_sonoma=['Sonoma Coast', 'Chalk Hill', 'Sonoma Valley', 'Bennett Valley', 'Mendocino Ridge', 'Russian River Valley', 'Sonoma Coast', 'Carneros', 'Alexander Valley', 'Dry Creek Valley', 'Sonoma County', 'Sonoma Mountain']
wanted_yakima=['Yakima Valley','Horse Heaven Hills','Snipes Mountains', 'Red Mountain']
wanted_walla=['Walla Walla Valley']
wanted_columbia=['Columbia Valley']
wanted_will=['Willamette Valley', 'Dundee Hills', 'Eola-Amity Hills', 'Ribbon Ridge', 'Chehalem Mountains']
wanted_others= wanted_napa +wanted_santa+wanted_sonoma+wanted_yakima+wanted_walla+wanted_columbia+wanted_will
for value in white_USA_df['appellation']:
    if value in wanted_napa: 
        a.append(value)
    if value in wanted_santa: 
        b.append(value)
    if value in wanted_sonoma: 
        c.append(value)
    if value in wanted_yakima: 
        d.append(value)
    if value in wanted_walla: 
        e.append(value)
    if value in wanted_columbia: 
        f.append(value)
    if value in wanted_will: 
        g.append(value)
    if value not in wanted_others: 
        white_USA_df.drop(white_USA_df.loc[white_USA_df['appellation']==value].index, inplace= True)
        h.append(value)
        
print('Napa Valley')
print(len(a))

print('Santa Cruz Mountains')
print(len(b))

print('Sonoma Coast')
print(len(c))

print('Yakima Valley')
print(len(d))

print('Walla Walla')
print(len(e))

print('Columbia')
print(len(f))

print('Willamette Valley')
print(len(g))
        

print('Others')
print(len(h))
print(h)

Napa Valley
339
Santa Cruz Mountains
21
Sonoma Coast
364
Yakima Valley
3
Walla Walla
0
Columbia
18
Willamette Valley
31
Others
8
['California', 'Sierra Foothills', 'California', 'California', 'California', 'California', 'Mendocino County', 'California']


#### Determine number of instances for each State

In [16]:
cali=['California']
oregon=['Oregon']
wash=['Washington']
total=['California','Oregon', 'Washington']
x=[]
y=[]
z=[]
xyz=[]

for value in white_USA_df['regions']:
    if value in cali:
        x.append(value)
    if value in oregon:
        y.append(value)
    if value in wash:
        z.append(value)
    if value not in total:
        xyz.append(value)
print('California')
print(white_USA_df['regions'].str.count('California').sum())
print('Oregon')
print(white_USA_df['regions'].str.count('Oregon').sum())
print('Washington')
print(white_USA_df['regions'].str.count('Washington').sum())
print("Other")
print(len(xyz))

California
724
Oregon
31
Washington
21
Other
0


#### Determine the columns that have the most NA values

In [17]:
print(white_USA_df['classification'].isnull().sum())
print(white_USA_df['avgPrcpJanuary'].isnull().sum())
print(white_USA_df['avgPrcpJune'].isnull().sum())
print(white_USA_df['avgPrcpSeptember'].isnull().sum())
print(white_USA_df['avgPrcpOctober'].isnull().sum())
print(white_USA_df['avgPrcpNovember'].isnull().sum())
print(white_USA_df['avgPrcpNovember'].isnull().sum())

776
24
399
621
621
621
621


#### Statistics for White Wine

In [18]:
print(st.mean(white_USA_df['score']))
print(st.median(white_USA_df['score']))
print(st.variance(white_USA_df['score']))
print(st.stdev(white_USA_df['score']))

90.94649484536083
91.18
8.24519518190888
2.871444789980974


#### Drop columns that have several NA values

In [19]:
red_USA_df.drop(columns=['wine_type', 'classification', 'avgPrcpJanuary', 'avgTempJanuary', 'avgPrcpNovember', 'avgTempNovember', 'avgPrcpDecember', 'avgTempDecember'], inplace=True)
white_USA_df.drop(columns=['classification', 'avgPrcpJanuary', 'avgTempJanuary', 'avgPrcpNovember', 'avgTempNovember', 'avgPrcpDecember', 'avgTempDecember'], inplace=True)
white_USA_df.dropna(inplace=True)
red_USA_df.dropna(inplace=True)
red_USA_df.to_csv('Red_Wine_Cleaned.csv')
white_USA_df.to_csv('White_Wine_Cleaned.csv')

In [20]:
white_USA_df

,wine,wine_id,appellation,color,wine_type,regions,country,vintage,is_primeurs,score,confidence_index,journalist_count,avgPrcpFebruary,avgTempFebruary,avgPrcpMarch,avgTempMarch,avgPrcpApril,avgTempApril,avgPrcpMay,avgTempMay,avgPrcpJune,avgTempJune,avgPrcpJuly,avgTempJuly,avgPrcpAugust,avgTempAugust,avgPrcpSeptember,avgTempSeptember,avgPrcpOctober,avgTempOctober
10,"Marcassin, Lorenzo Vineyard Chardonnay, White,...",101640,Sonoma Coast,White,dry,California,Usa,1998,False,91.74,B,4,0.671795,57,0.074516,64,0.084615,68,0.125806,67,0.005714,76,0.0,0,0.0,91,0.001667,83,0.038636,73
15,"Peter Michael Winery, Point Rouge Chardonnay, ...",114834,Sonoma County,White,dry,California,Usa,2015,False,95.69,C+,3,0.000000,0,0.000000,0,0.000000,0,0.000000,0,0.000000,0,0.0,0,0.0,0,0.000000,0,0.000000,0
20,"Peter Michael Winery, Belle Cote Chardonnay, W...",114819,Sonoma County,White,dry,California,Usa,2016,False,94.12,B,3,0.000000,0,0.000000,0,0.000000,0,0.000000,0,0.000000,0,0.0,0,0.0,0,0.000000,0,0.000000,0
21,"Peter Michael Winery, Mon Plaisir Chardonnay, ...",114832,Sonoma County,White,dry,California,Usa,2016,False,94.12,B,3,0.000000,0,0.000000,0,0.000000,0,0.000000,0,0.000000,0,0.0,0,0.0,0,0.000000,0,0.000000,0
25,"Peter Michael Winery, Mon Plaisir Chardonnay, ...",114832,Sonoma County,White,dry,California,Usa,2015,False,93.96,C+,3,0.000000,0,0.000000,0,0.000000,0,0.000000,0,0.000000,0,0.0,0,0.0,0,0.000000,0,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,"Truchard Vineyards, Chardonnay, White, Carneros",136966,Carneros,White,dry,California,Usa,1996,False,87.68,C,3,0.317959,62,0.104400,68,0.116333,71,0.061724,77,0.000000,85,0.0,90,0.0,90,0.006071,83,0.022759,77
779,"Peter Michael Winery, Belle Cote Chardonnay, W...",114819,Sonoma County,White,dry,California,Usa,1996,False,91.76,C+,3,0.000000,0,0.000000,0,0.000000,0,0.000000,0,0.000000,0,0.0,0,0.0,0,0.000000,0,0.000000,0
781,"Kistler Vineyards, Hudson Vineyard Chardonnay,...",91298,Carneros,White,dry,California,Usa,1995,False,93.90,A,3,0.032113,64,0.429811,63,0.044333,68,0.060968,72,0.037143,80,0.0,88,0.0,89,0.000000,84,0.000000,80
782,"Joseph Phelps Vineyards, Carneros Chardonnay, ...",89562,Los Carneros,White,dry,California,Usa,1996,False,86.86,A,3,0.375918,52,0.094000,54,0.127000,56,0.062414,60,0.000000,64,0.0,66,0.0,66,0.001071,63,0.016897,60
